In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [0]:
spark = SparkSession.builder.appName("silver_resident_shortterm").getOrCreate()

In [0]:
resident_df = spark.table("workspace.growth_poc.bronze_residents_shortterm")

In [0]:
resident_df.printSchema()

In [0]:
from pyspark.sql.functions import from_json, explode_outer
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType

# since the data is currently JSON string, 
# I need to convert it back to JSON object using from_json
resident_schema = StructType([
    StructField("국적지역", StringType(), False),
    StructField("년", IntegerType(), False),
    StructField("단기체류외국인 수", IntegerType(), False)  
])

 
# using from_json, the column `data` (JSON string) is converted into ArrayType(StructType)
# Each element is now a struct mapping the original JSON object's key-value pairs.
resident_df = resident_df.withColumn("data_parsed", from_json(col("data"), ArrayType(resident_schema)))

# flatten the array
exploded_df = resident_df.select(explode_outer("data_parsed"))

# convert each item in struct as a column
processed_df = exploded_df.select(
    col("col.국적지역").alias("Nationality"),
    col("col.년").alias("Year"),
    col("col.`단기체류외국인 수`").alias("Amount")
)

In [0]:
# replace vague nationality with null
exclude_list = [
    "기타", "국제연합", "미등록국가", "교황청",
    "영국속국민","영국속령지시민","영국외지민","영국외지시민","영국해외영토시민",
    "한국계러시아인","한국계중국인"
]

from pyspark.sql.functions import when

cleaned_df = processed_df.select(
    when(col("Nationality").isin(exclude_list), "None").otherwise(col("Nationality")).alias("Nationality")
    , "Year"
    , "Amount"
)

In [0]:
# 
replacement_dict = {
    # 러시아 관련
    "러시아(연방)": "러시아",
    "러시아연방": "러시아",
    
    # 남수단
    "남수단공화국": "남수단",
    
    # 그루지야 / 조지아
    "그루지야": "조지아",
    
    # 프랑스령 가이아나
    "불령가이아나": "프랑스령 가이아나",
    
    # 세르비아몬테네그로 → 세르비아 (혹은 정책에 따라 별도)
    "세르비아몬테네그로": "세르비아",
    
    # 러시아/중국/홍콩 관련
    "홍콩거주난민": "홍콩",
    
    # 예시: 긴 명칭 간소화
    "남아프리카공화국": "남아프리카",
    "도미니카공화국": "도미니카",
    "티모르민주공화국": "티모르",
    
    # 기타 소형 섬 / 특수 영토 간소화
    "미국인근섬": "미국",
    "마르티니크": "프랑스",
    "마카오": "중국",
    "세인트루시아": "세인트루시아",  # 유지
    "세인트빈센트그레나딘": "세인트빈센트",  # 축약 가능
    "세인트크리스토퍼네비스": "세인트키츠네비스",  # 통일 가능
    
    # 오타/변형
    "그리스": "그리스",
    "러시아": "러시아",
    "조지아": "조지아",
}

from pyspark.sql.functions import sum
cleaned_df = cleaned_df.replace(replacement_dict, subset = ['Nationality'])
cleaned_df = cleaned_df.groupBy(["Nationality", "Year"]).agg(
    sum("Amount").alias("Amount")
)

In [0]:
cleaned_df.orderBy("Nationality", "Year").show()

In [0]:
%pip install pycountry

In [0]:
import pycountry

nationalities = [row.Nationality for row in processed_df.select("Nationality").collect()]

for country_name in nationalities:
    # 조회할 국가명을 입력합니다.
    # country_name = "South Korea" # 또는 "대한민국" 등

    # 국가명으로 국가 객체를 가져옵니다.
    # (대소문자 구분 없이는 .get() 메서드에 lower() 또는 upper()를 적용할 수 있습니다)
    try:
        country = pycountry.countries.get(name=country_name)
        if country:
            # 2자리 국가 코드 (예: KR)
            alpha_2_code = country.alpha_2
            print(f"{country_name}의 2자리 국가 코드: {alpha_2_code}")

            # 3자리 국가 코드 (예: KOR)
            alpha_3_code = country.alpha_3
            print(f"{country_name}의 3자리 국가 코드: {alpha_3_code}")

            # 숫자 코드 (예: 410)
            numeric_code = country.numeric
            print(f"{country_name}의 숫자 코드: {numeric_code}")
        else:
            print(f"'{country_name}'에 해당하는 국가를 찾을 수 없습니다.")
    except Exception as e:
        print(f"국가 코드를 조회하는 중 오류가 발생했습니다: {e}")

    # 다른 국가명으로 시도해 볼 수 있습니다.
    # country_name_kr = "대한민국"
    # try:
    #     country_kr = pycountry.countries.get(common_name=country_name_kr) # 또는 name으로 시도
    #     if country_kr:
    #         print(f"{country_name_kr}의 2자리 국가 코드: {country_kr.alpha_2}")
    #     else:
    #             print(f"'{country_name_kr}'에 해당하는 국가를 찾을 수 없습니다.")
    except Exception as e:
        print(f"국가 코드를 조회하는 중 오류가 발생했습니다: {e}")